# Predicting Job Function

We are trying to predict the industry a job is in, instead of the companuy. For example a software engineer at Netlix would have:

- Compan: Netflix, Industry: Media. 
- Job: Software Engineer, Job Function: Software & IT

We will make this prediction in a similar manner as to industry, using the ollama API, giving the API three categories of data, skills, responsabilities, and job description as well as the actual job title. 

From this information we think the moidel should be able to correctly predict the job function. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import os
import time

In [4]:
df = pd.read_csv('mapped_sectors.csv')

In [5]:
df

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Sector,Industry,Real-Salary-Industry
0,579575048679091,3 to 9 Years,MCA,$59K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,126334,...,Wireless Network Engineer,USAJOBS,"Wireless Network Engineers design, implement, ...","{'Flexible Spending Accounts (FSAs), Relocatio...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Lendlease Group,Real Estate/Construction,Real Estate,Real Estate
1,288941837754490,4 to 15 Years,B.Tech,$57K-$117K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,35869,...,Urban Planner,The Muse,Develop and implement urban planning strategie...,"{'Employee Referral Programs, Financial Counse...",Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",United Technologies Corporation,Aerospace/Building Systems,Aerospace and Defense,Aerospace
2,2802376880223274,0 to 9 Years,B.Tech,$63K-$90K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,80163,...,UX Strategist,SimplyHired,"Develop user experience (UX) strategies, creat...","{'Transportation Benefits, Professional Develo...",User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,International Flavors & Fragrances,Flavors & Fragrances,Chemicals,Consumer Goods
3,2393222411935908,5 to 11 Years,MCA,$55K-$124K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,79017,...,Benefits Coordinator,Internships.com,Benefits Coordinators administer employee bene...,"{'Transportation Benefits, Professional Develo...",Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",KeyCorp,Financial Services,Commercial Banks,Finance
4,2402523179167443,0 to 10 Years,MCA,$65K-$91K,"Washington, D.C.",USA,37.0902,-95.7129,Intern,65144,...,Environmental Designer,USAJOBS,"Design outdoor spaces, parks, and landscapes w...","{'Childcare Assistance, Paid Time Off (PTO), R...",Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,Exelon,Energy,Utilities: Gas and Electric,Energy and Mining
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7386,1248373472037280,4 to 8 Years,MBA,$61K-$121K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,14184,...,Child Welfare Worker,Internships.com,"Work with at-risk children and families, asses...","{'Childcare Assistance, Paid Time Off (PTO), R...",Child welfare policies and regulations Child p...,Provide support and counseling to children and...,Intuit,Financial Technology,Computer Software,Finance
7387,481574301429114,5 to 9 Years,PhD,$57K-$103K,"Washington, D.C.",USA,37.0902,-95.7129,Temporary,104378,...,Data Scientist,Snagajob,Data Scientists use their expertise in data an...,"{'Employee Assistance Programs (EAP), Tuition ...",Machine learning algorithms and libraries (e.g...,Apply machine learning algorithms and statisti...,Coty Inc.,Beauty/Cosmetics,Consumer Products - Cosmetics,Consumer Goods
7388,639794127037359,3 to 8 Years,MCA,$59K-$89K,"Washington, D.C.",USA,37.0902,-95.7129,Part-Time,70360,...,Media Relations Specialist,Indeed,Media Relations Specialists manage relationshi...,"{'Tuition Reimbursement, Stock Options or Equi...",Media relations Press releases Media outreach ...,Manage relationships with media outlets and jo...,Macy's,Retail,General Merchandisers,Consumer Goods
7389,1394850457865204,3 to 10 Years,BCA,$64K-$115K,"Washington, D.C.",USA,37.0902,-95.7129,Contract,81625,...,Healthcare Business Analyst,LinkedIn,Healthcare Business Analysts work in the healt...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Healthcare industry knowledge Health data anal...,"Work in the healthcare sector, analyzing healt...",Origin Energy,Energy/Oil and Gas,Energy,Energy and Mining


In [39]:
df.columns

Index(['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location',
       'Country', 'latitude', 'longitude', 'Work Type', 'Company Size',
       'Job Posting Date', 'Preference', 'Contact Person', 'Contact',
       'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits',
       'skills', 'Responsibilities', 'Company', 'Sector', 'Industry',
       'Real-Salary-Industry'],
      dtype='object')

In [46]:
df2 = pd.DataFrame()
df2["Role"] = df["Role"]
df2["Skills"] = df["skills"]
df2["Responsibilities"] = df["Responsibilities"]
df2["Job Description"] = df["Job Description"]
df2['Real-Salary-Industry'] = df['Real-Salary-Industry']

df2

,Role,Skills,Responsibilities,Job Description,Real-Salary-Industry
0,Wireless Network Engineer,Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...","Wireless Network Engineers design, implement, ...",Real Estate
1,Urban Planner,Urban design and planning Zoning regulations G...,"Plan and design urban spaces, parks, and recre...",Develop and implement urban planning strategie...,Aerospace
2,UX Strategist,User experience strategy User journey mapping ...,Develop UX strategies and roadmaps based on us...,"Develop user experience (UX) strategies, creat...",Consumer Goods
3,Benefits Coordinator,Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",Benefits Coordinators administer employee bene...,Finance
4,Environmental Designer,Environmental design principles Sustainability...,Focus on environmentally sustainable landscape...,"Design outdoor spaces, parks, and landscapes w...",Energy and Mining
...,...,...,...,...,...
7386,Child Welfare Worker,Child welfare policies and regulations Child p...,Provide support and counseling to children and...,"Work with at-risk children and families, asses...",Finance
7387,Data Scientist,Machine learning algorithms and libraries (e.g...,Apply machine learning algorithms and statisti...,Data Scientists use their expertise in data an...,Consumer Goods
7388,Media Relations Specialist,Media relations Press releases Media outreach ...,Manage relationships with media outlets and jo...,Media Relations Specialists manage relationshi...,Consumer Goods
7389,Healthcare Business Analyst,Healthcare industry knowledge Health data anal...,"Work in the healthcare sector, analyzing healt...",Healthcare Business Analysts work in the healt...,Energy and Mining


In [40]:
# Get uniquq roles
roles = df2["Role"].unique()
print(roles)

['Wireless Network Engineer' 'Urban Planner' 'UX Strategist'
 'Benefits Coordinator' 'Environmental Designer'
 'User Experience Designer' 'Front-End Developer' 'Wedding Planner'
 'Residential Interior Designer' 'Fine Arts Instructor'
 'Database Administrator' 'Research Psychologist'
 'Sales Account Executive' 'Environmental Impact Analyst'
 'Print Graphic Designer' 'UI/UX Developer' 'Automation Test Engineer'
 'Investment Portfolio Manager' 'Sales Account Manager'
 'Quality Control Analyst' 'Lighting Designer'
 'Client Relationship Manager' 'Industrial Engineer' 'Family Counselor'
 'Content Creator' 'Financial Analyst' 'Deliverability Analyst'
 'Test Automation Engineer' 'Subject Matter Expert'
 'Service Quality Assurance Manager' 'Environmental Consultant'
 'Litigation Support Specialist' 'Event Coordinator'
 'Special Education Teacher' 'Accounting Manager' 'Litigation Paralegal'
 'Analytical Chemist' 'SEM Analyst' 'Systems Administrator'
 'CAD Technician' 'Paralegal' 'Personal Secret

# Preparing the data for the API

Here we remove stop words and get rid of null values

In [8]:
from sklearn.feature_extraction import text

stop = text.ENGLISH_STOP_WORDS

for i in stop:
    df2 = df2.replace(to_replace=r"\b%s\b"%i, value="", regex=True)
df2.head()

,Role,Skills,Responsibilities,Job Description
0,Wireless Network Engineer,Wireless network design architecture Wi-Fi st...,"Design, configure, optimize wireless networks...","Wireless Network Engineers design, implement, ..."
1,Urban Planner,Urban design planning Zoning regulations GIS ...,"Plan design urban spaces, parks, recreationa...","Develop implement urban planning strategies, ..."
2,UX Strategist,User experience strategy User journey mapping ...,Develop UX strategies roadmaps based user re...,"Develop user experience (UX) strategies, creat..."
3,Benefits Coordinator,Employee benefits administration Benefits comp...,"Administer employee benefits programs, respond...",Benefits Coordinators administer employee bene...
4,Environmental Designer,Environmental design principles Sustainability...,Focus environmentally sustainable landscape d...,"Design outdoor spaces, parks, landscapes em..."


In [9]:
# Check for missing values
df2.isnull().sum()

Role                0
Skills              0
Responsibilities    0
Job Description     0
dtype: int64

# Prepare the API

Setting up the API call function and mathod calling

In [10]:
real_salary_sectors = [
    "Aerospace",
    "Construction",
    "Consumer Goods",
    "Coorporate Services",
    "Defense",
    "Design",
    "Education",
    "Energy and Mining",
    "Engineering",
    "Entertainment",
    "Finance",
    "Hardware and Newtowrking",
    "Healthcare",
    "Manufacturing",
    "Media and Communications",
    "Nonprofit",
    "Public Administration",
    "Real Estate",
    "Recreation and Travel",
    "Retail",
    "Software and IT Services",
    "Telecommunications",
    "Transportation and Logistics",
    "Wellness and Fitness"
]

In [60]:
def getAttributes(job):
    index = df2[df2["Role"].str.contains(job, na=False)].iloc[0]
    skills = index["Skills"]
    responsibilities = index["Responsibilities"]
    description = index["Job Description"]
    return " Skills: " + skills + " Responsibilities: " + responsibilities + " Job Description: " + description

In [69]:
print(getAttributes("Data Scientist"))

 Skills: Machine learning algorithms and libraries (e.g., scikit-learn, TensorFlow, PyTorch) Statistical analysis and modeling Data preprocessing and cleaning Big data technologies (e.g., Hadoop, Spark) Data visualization Strong programming skills (Python, R) Responsibilities: Apply machine learning algorithms and statistical techniques to extract insights from large datasets. Develop predictive models. Conduct data experiments and A/B testing. Job Description: Data Scientists use their expertise in data analysis and machine learning to extract valuable insights and predictions from data. They build and train models, conduct statistical analysis, and communicate findings to drive data-driven decision-making in the organization.


In [11]:
import requests
url = "http://localhost:11434/api/chat"

In [62]:
def getJobFunction(prompt):
    real_salary_sectors_str = ", ".join(real_salary_sectors)
    attributes_str = getAttributes(prompt)
    data = {
        "model": "llama3",
        "messages": [
            {
                "role": "user",
                "content": f"""For the job: {prompt},
                            {attributes_str} provides you some attributes of the job: skills, responsibilities, and job description. This is given to help you assign the job function.
                            For position: {prompt}. assign a job function from this list {real_salary_sectors_str}. 
                            Do not add any other commentary to your response only the job function."""
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()["message"]["content"]
    
    return response

In [66]:
print(getJobFunction("Data Scientist"))

Finance


In [67]:
print(getJobFunction("Urban Planner"))

Transportation and Logistics


In [68]:
print(getJobFunction("Benefits Coordinator"))

Finance


# Testing the model

Testing the model with the same random selection approach, however this will be validated using the sector feature

In [73]:
test_sizes = [20]
times = []

In [74]:
import random
import time

for test_size in test_sizes:
    print(f"Testing with {test_size} companies")
    print("=====================================")
    start_time = time.time()
    # Pick ___ random roles
    random_role = random.sample(list(df2["Role"]), test_size)
    # Get the job function for each role
    job_functions = []
    roles_dict = {}
    sectors = []
    print("Predicting job functions for role...")
    for role in random_role:
        # Make the prediction
        job_function = getJobFunction(role)
        print(f"{role} -> {job_function}")
        # Append the prediction to the industry list
        job_functions.append(job_function)
        # Add the company and the prediction to the dictionary
        roles_dict[role] = job_function
    end_time = time.time()
    times.append(end_time - start_time)
    print("=====================================")
    print(f"Number of roles: {test_size}")
    print(f"Time taken: {end_time - start_time} seconds")

Testing with 20 companies
Predicting job functions for role...
Personal Assistant -> Corporate Services
UI/UX Front-End Developer -> Software and IT Services
Studio Art Teacher -> Education
Portfolio Manager -> Finance
Territory Sales Manager -> Manufacturing
Architectural Drafter -> Construction
Market Researcher -> Consumer Goods
Network Engineer -> Telecommunications
Cloud Systems Engineer -> Software and IT Services
Digital Marketing Coordinator -> Media and Communications
Senior Researcher -> Engineering
Research Chemist -> Engineering
Wedding Consultant -> Entertainment
Paralegal -> Finance
Quality Control Engineer -> Manufacturing
Network Security Engineer -> Software and IT Services
Executive Assistant -> Finance
Outside Sales Representative -> Hardware and Networking
System Administrator -> Software and IT Services
Employee Development Manager -> Software and IT Services
Number of roles: 20
Time taken: 57.23338484764099 seconds


In [76]:
df_comparison = pd.DataFrame()
df_comparison["Role"] = random_role
df_comparison["Predicted Job Function"] = job_functions

df_comparison

,Role,Predicted Job Function
0,Personal Assistant,Corporate Services
1,UI/UX Front-End Developer,Software and IT Services
2,Studio Art Teacher,Education
3,Portfolio Manager,Finance
4,Territory Sales Manager,Manufacturing
5,Architectural Drafter,Construction
6,Market Researcher,Consumer Goods
7,Network Engineer,Telecommunications
8,Cloud Systems Engineer,Software and IT Services
9,Digital Marketing Coordinator,Media and Communications
